Create Libraries from a set of building block csv input files:<br>
-BuildingBlocks.csv (BB smiles and BB IDs) <br>
-Library BB file   (The BBs to use for each elemement in a library. Located in "Library BB Groups" folder)<br><br>

Library assembly is performed as a virtual analogue of the series of synthetic transformtions that would be used to crerate the library in real life.<br>
Synthetic steps are performed starting with the DNA attachement point. Here, a methyl carboxamide is used as the "mock" DNA. <br><br>
The virtual synthesis is carried out according to a specfically formatted input string in the form:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|A|\<RxnID\>|B|\<RxnID\>|C|...<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;where:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>|X|</b> denotes an ordered set of BBs to be used at the given position<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>\<RxnID\></b> denotes a specific reaction protocol to be iteratively applied to the BB sets on each side.<br><br>

Moving left to right, each additional reaction is performed on the growing synthetic intermediate from the preceeding steps.<br>
More formally, the string "<b>|A|\<R2\>|B|\<R2\>|C|\<R1\>|D|</b>" is interpreted as the following three-step reaction series: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|A|\<R2\>|B|<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|[AB]|\<R2\>|C|<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|ABC|\<R1\>|D|<br><br>

Reaction IDs used in this notebook:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;R1- Amide bond formation<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;R2 - Amide bond formation followed by Boc deprotection<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;R3 - Amide bond formation followed by Fmoc deprotection<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;R4 - Intramolecular amide bond formation<br>


In [ ]:
from Modules import SetGenerators
from Modules import DELTARxns 
import pandas as pd

class CombinedOutput:
    def __init__(self):
        self.combined_df=pd.DataFrame()
    def Add_Output(self, Data):
        self.combined_df=pd.concat([self.combined_df,Data], axis=0)
        
AllData=CombinedOutput()


AB - 2 Cycle Linear 

In [ ]:
#USER INPUTS
DELTARxns.complex_string='|A|<R1>|B|'
DELTARxns.DELname='AB'

DEL=SetGenerators.BBset('Library BB Groups/AB_L.csv')
DELTARxns.BB_IDs_path= "BuildingBlocks.csv"
output_path="Notebook Outputs/AB_L_Output.csv"


#Generate the BB sets 
DELTARxns.Random_df=DEL.make_random_set(1000)

#Map the reagents to thier intended reactivity type (Amine=NH, Carboxylic Acid=CA)
DELTARxns.type_dict={'A':'A_NH','B':'B_CA','C':'C_CA','D': 'D_CA','Z':'Int_NH'}

#Class object creation for each reaction
global temp_prod
DELTARxns.products=DELTARxns.rxn_products()
DELTARxns.products2=DELTARxns.rxn_products()
DELTARxns.temp_prod=DELTARxns.temp_products()

AB_L_output=DELTARxns.run_reaction()

AB_L_output.to_csv(output_path)
AllData.Add_Output(AB_L_output)

Ab(C) - 2 Cycle Branched 

In [ ]:
#USER INPUTS
DELTARxns.complex_string='|A|<R2>|B|<R1>|C|'
DELTARxns.DELname='AbC'

DEL=SetGenerators.BBset('Library BB Groups/AB_B.csv')
DELTARxns.BB_IDs_path= "BuildingBlocks.csv"
output_path="Notebook Outputs/AB_B_Output.csv"


#Generate the BB sets 
DELTARxns.Random_df=DEL.make_random_set(1000)

#Map the reagents to thier intended reactivity type (Amine=NH, Carboxylic Acid=CA)
DELTARxns.type_dict={'A':'A_NH','B':'B_CA','C':'C_CA','D': 'D_CA','Z':'Int_NH'}

#Class object creation for each reaction
global temp_prod
DELTARxns.products=DELTARxns.rxn_products()
DELTARxns.products2=DELTARxns.rxn_products()
DELTARxns.temp_prod=DELTARxns.temp_products()

AB_B_output=DELTARxns.run_reaction()

AB_B_output.to_csv(output_path)
AllData.Add_Output(AB_B_output)

!AbC! - 2 Cycle Cyclic

In [ ]:
#USER INPUTS
DELTARxns.complex_string='|A|<R2>|B|<R2>|C|<R2>|D|'
DELTARxns.DELname='!AbC!'

DEL=SetGenerators.BBset('Library BB Groups/AB_C.csv')
DELTARxns.BB_IDs_path= "BuildingBlocks.csv"
output_path="Notebook Outputs/CyclicAbC_Output.csv"



#Generate the BB sets 
DELTARxns.Random_df=DEL.make_random_set(1000)

#Map the reagents to thier intended reactivity type (Amine=NH, Carboxylic Acid=CA)
DELTARxns.type_dict={'A':'A_NH','B':'B_CA','C':'C_CA','D': 'D_CA','Z':'Int_NH'}

#Class object creation for each reaction
global temp_prod
DELTARxns.products=DELTARxns.rxn_products()
DELTARxns.products2=DELTARxns.rxn_products()
DELTARxns.temp_prod=DELTARxns.temp_products()

output_df=DELTARxns.run_reaction()

global IDs
IDs=DELTARxns.prod_dict.keys()
output_df['PreCycle']=DELTARxns.final_deprotection(output_df['Smiles'], PG='ester')
precyle_list=output_df['PreCycle'].values.tolist()
output_df['Cycle']=DELTARxns.R4(precyle_list)

CyclicAbC_output_df=output_df.loc[:, ['DEL','ID']]
CyclicAbC_output_df['Smiles']=output_df['Cycle']
output_df.to_csv(output_path)
AllData.Add_Output(CyclicAbC_output_df)

ABC - 3 Cycle Linear

In [ ]:
#USER INPUTS
DELTARxns.complex_string='|A|<R2>|B|<R1>|C|'
DELTARxns.DELname='ABC'

DEL=SetGenerators.BBset('Library BB Groups/ABC_L.csv')
DELTARxns.BB_IDs_path= "BuildingBlocks.csv"
output_path="Notebook Outputs/ABC_L_Output.csv"



#Generate the BB sets 
DELTARxns.Random_df=DEL.make_random_set(1000)

#Map the reagents to thier intended reactivity type (Amine=NH, Carboxylic Acid=CA)
DELTARxns.type_dict={'A':'A_NH','B':'B_CA','C':'C_CA','D': 'D_CA','Z':'Int_NH'}

#Class object creation for each reaction
global temp_prod
DELTARxns.products=DELTARxns.rxn_products()
DELTARxns.products2=DELTARxns.rxn_products()
DELTARxns.temp_prod=DELTARxns.temp_products()

ABC_output_df=DELTARxns.run_reaction()

ABC_output_df.to_csv(output_path)
AllData.Add_Output(ABC_output_df)

AB(C) - 3 Cycle Branched

In [ ]:
#USER INPUTS
DELTARxns.complex_string='|A|<R3>|B|<R1>|C|'
DELTARxns.DELname='AB(C)'

DEL=SetGenerators.BBset('Library BB Groups/ABC_B.csv')
DELTARxns.BB_IDs_path= "BuildingBlocks.csv"
output_path="Notebook Outputs/ABC_B_Output.csv"



#Generate the BB sets 
DELTARxns.Random_df=DEL.make_random_set(1000)

#Map the reagents to thier intended reactivity type (Amine=NH, Carboxylic Acid=CA)
DELTARxns.type_dict={'A':'A_NH','B':'B_CA','C':'C_CA','D': 'D_CA','Z':'Int_NH'}

#Class object creation for each reaction
global temp_prod
DELTARxns.products=DELTARxns.rxn_products()
DELTARxns.products2=DELTARxns.rxn_products()
DELTARxns.temp_prod=DELTARxns.temp_products()

ABC_B_output_df=DELTARxns.run_reaction()

ABC_B_output_df.to_csv(output_path)
AllData.Add_Output(ABC_B_output_df)

!ABC! - 3 Cycle Cyclic

In [ ]:
#USER INPUTS
DELTARxns.complex_string='|A|<R2>|B|<R2>|C|<R2>|D|'
DELTARxns.DELname='!ABC!'

DEL=SetGenerators.BBset('Library BB Groups/ABC_C.csv')
DELTARxns.BB_IDs_path= "BuildingBlocks.csv"
output_path="Notebook Outputs/ABC_C_Output.csv"



#Generate the BB sets 
DELTARxns.Random_df=DEL.make_random_set(1000)

#Map the reagents to thier intended reactivity type (Amine=NH, Carboxylic Acid=CA)
DELTARxns.type_dict={'A':'A_NH','B':'B_CA','C':'C_CA','D': 'D_CA','Z':'Int_NH'}

#Class object creation for each reaction
global temp_prod
DELTARxns.products=DELTARxns.rxn_products()
DELTARxns.products2=DELTARxns.rxn_products()
DELTARxns.temp_prod=DELTARxns.temp_products()

output_df=DELTARxns.run_reaction()

global IDs
IDs=DELTARxns.prod_dict.keys()
output_df['PreCycle']=DELTARxns.final_deprotection(output_df['Smiles'], PG='ester')
precyle_list=output_df['PreCycle'].values.tolist()
output_df['Cycle']=DELTARxns.R4(precyle_list)

ABC_C_output_df=output_df.loc[:,['DEL','ID']]
ABC_C_output_df['Smiles']=output_df['Cycle']
output_df.to_csv(output_path)
AllData.Add_Output(ABC_C_output_df)

ABCD - 4 Cycle Linear

In [ ]:
#USER INPUTS

DELTARxns.complex_string='|A|<R2>|B|<R2>|C|<R1>|D|'
DELTARxns.DELname='ABCD'

DEL=SetGenerators.BBset('Library BB Groups/ABCD_L.csv')
DELTARxns.BB_IDs_path= "BuildingBlocks.csv"
output_path="Notebook Outputs/ABCD_L_Output.csv"

#Generate the BB sets 
DELTARxns.Random_df=DEL.make_random_set(1000)

#Map the reagents to thier intended reactivity type (Amine=NH, Carboxylic Acid=CA)
DELTARxns.type_dict={'A':'A_NH','B':'B_CA','C':'C_CA','D': 'D_CA','Z':'Int_NH'}

#Class object creation for each reaction
global temp_prod
DELTARxns.products=DELTARxns.rxn_products()
DELTARxns.products2=DELTARxns.rxn_products()
DELTARxns.temp_prod=DELTARxns.temp_products()

ABCD_output_df=DELTARxns.run_reaction()

ABCD_output_df.to_csv(output_path)
AllData.Add_Output(ABCD_output_df)

In [ ]:
AllData.combined_df.to_csv('Notebook Outputs/Combined_DELs.csv')